# 1. Postal Codes, Canada

1. First table from wiki-page 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' was read into the dataframe df.

In [97]:
import pandas as pd

tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = pd.DataFrame(tables[0])
df.rename(columns={'Postal code':'Postal Code'}, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


2. The rows with Borough value 'Not assigned' were removed from dataframe.

In [98]:
df = df[df['Borough'] != 'Not assigned']
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


3. No rows with Neighborhood value 'Not assigned' were found.

4. If one Postal code belongs to several neighborhoods, their names are separated with slash.

5. The shape of dataframe:

In [92]:
df.shape

(103, 3)

# 2. Coordinates

1. Geographical coordinates of each postal code from http://cocl.us/Geospatial_data loaded into the dataframe coords.

In [60]:
coords = pd.read_csv('http://cocl.us/Geospatial_data')
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


2. Tried to join coords with initial dataframe df using left join.

In [95]:
#df = df.join(coords, how='left', rsuffix='2').drop(columns=['Postal Code2'])
#df.head()

As after join operation there were NaN coordinates for some Postal Codes, joined df with coords manually:

In [100]:
for index, row in df.iterrows():     
    crd = coords[coords['Postal Code'] == row['Postal Code']]
    df.at[index, 'Latitude'] = crd.iat[0, 1]
    df.at[index, 'Longitude'] = crd.iat[0, 2]
  
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
5,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


3. The shape of dataframe:

In [103]:
df.shape

(103, 5)

# 3. Map of Toronto

1. Install Folium 

In [10]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

In [105]:
import folium

# create map of New York using latitude and longitude values
tmap = folium.Map(location=[43.7001, -79.4163], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(tmap)  
    
tmap

# 4. Explore Neighborhoods

1. Define Foursquare Credentials and Version

In [107]:
CLIENT_ID = 'W55WQMBNJ4LC1JXMYV112X0I3FSJPLBKBHHCBYBXC3GB30DZ' # your Foursquare ID
CLIENT_SECRET = 'C0CAN3C2INA2N2E3RPKKLAV3ALGGG2DAWCRDORWDBCT3DSRX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: W55WQMBNJ4LC1JXMYV112X0I3FSJPLBKBHHCBYBXC3GB30DZ
CLIENT_SECRET:C0CAN3C2INA2N2E3RPKKLAV3ALGGG2DAWCRDORWDBCT3DSRX


2. Explore the neighborhood with Postal Code M7A in dataframe

In [136]:
nb = df[df['Postal Code'] == 'M7A'].reset_index()

name = nb.loc[0, 'Neighborhood'] #neighborhood name 
lat = nb.loc[0, 'Latitude'] # neighborhood latitude value
lng = nb.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(name, 
                                                               lat, 
                                                               lng))

Latitude and longitude values of Queen's Park / Ontario Provincial Government are 43.6623015, -79.3894938.


3. Get the top 100 venues within a radius of 500 meters for selected neighborhood

In [153]:
import requests
from pandas.io.json import json_normalize

# create the API request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    500, 
    100)
            
# make the GET request
venues = requests.get(url).json()["response"]['groups'][0]['items']

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
# load venues into dataframe
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Queen's Park,Park,43.663946,-79.392180
1,Mercatto,Italian Restaurant,43.660391,-79.387664
2,Neo Coffee Bar,Coffee Shop,43.660140,-79.385870
3,Central YMCA,Distribution Center,43.663083,-79.385025
4,Starbucks,Coffee Shop,43.659456,-79.390411


In [155]:
nearby_venues.shape

(36, 4)

4. Get the top 100 venues within a radius of 500 meters for all neighborhoods

In [156]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

all_venues = getNearbyVenues(names=df['Neighborhood'],
                            latitudes=df['Latitude'],
                            longitudes=df['Longitude']
                            )

Parkwoods
Victoria Village
Regent Park / Harbourfront
Lawrence Manor / Lawrence Heights
Queen's Park / Ontario Provincial Government
Islington Avenue
Malvern / Rouge
Don Mills
Parkview Hill / Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Rouge Hill / Port Union / Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Thorncliffe Park
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Scarborough Village
Fairview / Henry Farm / Oriole
Northwood Park / York University
East Toronto
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East Birchmount Park
Bayview Village
Do

5. Check dataframe

In [159]:
print(all_venues.shape)
all_venues.head()

(2192, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


6. Check how many venues were returned for each neighborhood

In [160]:
all_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
Alderwood / Long Branch,10,10,10,10,10,10
Bathurst Manor / Wilson Heights / Downsview North,22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
Bedford Park / Lawrence Manor East,23,23,23,23,23,23
Berczy Park,56,56,56,56,56,56
Birch Cliff / Cliffside West,4,4,4,4,4,4
Brockton / Parkdale Village / Exhibition Place,22,22,22,22,22,22
Business reply mail Processing CentrE,16,16,16,16,16,16


# 4. Analyze Neighborhoods

In [162]:
# one hot encoding
onehot = pd.get_dummies(all_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = all_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


1. Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [163]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000
1,Alderwood / Long Branch,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000
2,Bathurst Manor / Wilson Heights / Downsview North,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.045455,0.000000,0.00,0.000000,0.000000,0.00000
3,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000
4,Bedford Park / Lawrence Manor East,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000
5,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000
6,Birch Cliff / Cliffside West,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000
7,Brockton / Parkdale Village / Exhibition Place,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000
8,Business reply mail Processing CentrE,0.062500,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000
9,CN Tower / King and Spadina / Railway Lands / ...,0.000000,0.0,0.000000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000


2. Print each neighborhood along with the top 5 most common venues

In [164]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge  0.25
1  Latin American Restaurant  0.25
2               Skating Rink  0.25
3             Breakfast Spot  0.25
4   Mediterranean Restaurant  0.00


----Alderwood / Long Branch----
          venue  freq
0   Pizza Place   0.2
1           Gym   0.1
2  Skating Rink   0.1
3  Dance Studio   0.1
4   Coffee Shop   0.1


----Bathurst Manor / Wilson Heights / Downsview North----
            venue  freq
0            Bank  0.09
1     Coffee Shop  0.09
2        Pharmacy  0.05
3   Deli / Bodega  0.05
4  Sandwich Place  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2  Japanese Restaurant  0.25
3   Chinese Restaurant  0.25
4          Yoga Studio  0.00


----Bedford Park / Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.09
1         Coffee Shop  0.09
2  Italian Restaurant  0.09
3          Restaurant  0.09
4            Phar

3. Display the top 10 venues for each neighborhood in new dataframe

In [168]:
import numpy as np

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Lounge,Breakfast Spot,Latin American Restaurant,Women's Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
1,Alderwood / Long Branch,Pizza Place,Gym,Skating Rink,Sandwich Place,Dance Studio,Pub,Pool,Coffee Shop,Pharmacy,Colombian Restaurant
2,Bathurst Manor / Wilson Heights / Downsview North,Bank,Coffee Shop,Diner,Ice Cream Shop,Supermarket,Deli / Bodega,Sushi Restaurant,Middle Eastern Restaurant,Restaurant,Pizza Place
3,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Women's Store
4,Bedford Park / Lawrence Manor East,Sandwich Place,Coffee Shop,Italian Restaurant,Restaurant,Juice Bar,Thai Restaurant,Liquor Store,Fast Food Restaurant,Indian Restaurant,Pub


# 5. Cluster Neighborhoods 

1. Run k-means to cluster the neighborhood into 5 clusters

In [173]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4], dtype=int32)

2. Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [176]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Food & Drink Shop,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Empanada Restaurant
3,M4A,North York,Victoria Village,43.725882,-79.315572,4.0,Coffee Shop,Pizza Place,Intersection,Portuguese Restaurant,Hockey Arena,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
4,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,4.0,Coffee Shop,Park,Pub,Bakery,Restaurant,Breakfast Spot,Mexican Restaurant,Theater,Café,Electronics Store
5,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,4.0,Clothing Store,Women's Store,Boutique,Furniture / Home Store,Event Space,Miscellaneous Shop,Coffee Shop,Vietnamese Restaurant,Accessories Store,Deli / Bodega
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,4.0,Coffee Shop,Diner,Gym,Creperie,Music Venue,Mexican Restaurant,Juice Bar,Italian Restaurant,Hobby Shop,Wings Joint


3. Visualize the resulting clusters

In [183]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[43.7001, -79.4163], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    try:
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[int(cluster)-1],
            fill_opacity=0.7).add_to(map_clusters)
    except:
        pass
       
map_clusters

4. Examine clusters

Cluster 1:

In [205]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Scarborough,0.0,Construction & Landscaping,Fast Food Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
89,North York,0.0,Baseball Field,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Women's Store,Event Space
169,Etobicoke,0.0,Construction & Landscaping,Baseball Field,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Women's Store,Department Store


Cluster 2:

In [206]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,North York,1.0,Park,Food & Drink Shop,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Empanada Restaurant
32,York,1.0,Park,Market,Women's Store,College Rec Center,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
57,East York,1.0,Park,Coffee Shop,Convenience Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store
77,North York,1.0,Park,Basketball Court,Bakery,Construction & Landscaping,Dumpling Restaurant,Drugstore,Donut Shop,Eastern European Restaurant,Doner Restaurant,Deli / Bodega
93,Central Toronto,1.0,Park,Bus Line,Swim School,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center,Dance Studio
98,York,1.0,Park,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant,Ethiopian Restaurant
100,North York,1.0,Park,Bank,Convenience Store,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
103,Central Toronto,1.0,Park,Jewelry Store,Sushi Restaurant,Trail,Bus Line,Drugstore,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
116,Etobicoke,1.0,Park,Pizza Place,Sandwich Place,Mobile Phone Shop,College Rec Center,College Gym,Drugstore,Donut Shop,Doner Restaurant,Dog Run
129,Central Toronto,1.0,Park,Summer Camp,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dance Studio


Cluster 3:

In [209]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Central Toronto,2.0,Garden,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dance Studio


Cluster 4:

In [207]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
73,North York,3.0,Cafeteria,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dance Studio


Cluster 5:

In [208]:
merged.loc[merged['Cluster Labels'] == 4, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,4.0,Coffee Shop,Pizza Place,Intersection,Portuguese Restaurant,Hockey Arena,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
4,Downtown Toronto,4.0,Coffee Shop,Park,Pub,Bakery,Restaurant,Breakfast Spot,Mexican Restaurant,Theater,Café,Electronics Store
5,North York,4.0,Clothing Store,Women's Store,Boutique,Furniture / Home Store,Event Space,Miscellaneous Shop,Coffee Shop,Vietnamese Restaurant,Accessories Store,Deli / Bodega
6,Downtown Toronto,4.0,Coffee Shop,Diner,Gym,Creperie,Music Venue,Mexican Restaurant,Juice Bar,Italian Restaurant,Hobby Shop,Wings Joint
11,North York,4.0,Gym,Coffee Shop,Restaurant,Beer Store,Japanese Restaurant,Supermarket,Concert Hall,Clothing Store,Chinese Restaurant,Caribbean Restaurant
12,East York,4.0,Pizza Place,Bank,Bus Line,Café,Gym / Fitness Center,Intersection,Pet Store,Pharmacy,Gastropub,Athletics & Sports
13,Downtown Toronto,4.0,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Café,Electronics Store,Italian Restaurant,Furniture / Home Store
14,North York,4.0,Pizza Place,Park,Pub,Japanese Restaurant,Donut Shop,Drugstore,Doner Restaurant,Dog Run,Distribution Center,Curling Ice
18,Scarborough,4.0,History Museum,Bar,Construction & Landscaping,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Women's Store
20,North York,4.0,Gym,Coffee Shop,Restaurant,Beer Store,Japanese Restaurant,Supermarket,Concert Hall,Clothing Store,Chinese Restaurant,Caribbean Restaurant
